In [ ]:
import os

os.environ["LEPHAREDIR"] = "/home/cohen/lsst/lephare/LEPHARE"
os.environ["LEPHAREWORK"] = "/home/cohen/lsst/lephare/LEPHARE/tests/WORK"

In [ ]:
from lephare import *
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
keymap = read_config("../examples/COSMOS.para")
keymap["ZPHOTLIB"] = keyword("ZPHOTLIB", "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS")
keymap["CAT_IN"] = keyword("CAT_IN", "$LEPHAREDIR/examples/COSMOS.in")
keymap["CAT_OUT"] = keyword("CAT_OUT", "zphot_short.out")
keymap["ADD_EMLINES"] = keyword("ADD_EMLINES", "0,100")
keymap["AUTO_ADAPT"] = keyword("ADD_EMLINES", "YES")
keymap["Z_STEP"] = keyword("Z_STEP", "0.04,0,6")
keymap["CAT_LINES"] = keyword("CAT_LINES", "1,100")
keymap["SPEC_OUT"] = keyword("SPEC_OUT", "YES")
keymap["PARA_OUT"] = keyword("PARA_OUT", "$LEPHAREDIR/examples/output.para")
keymap["VERBOSE"] = keyword("VERBOSE", "NO")
keymap["ZFIX"] = keyword("ZFIX", "NO")

In [ ]:
photz = PhotoZ(keymap);

In [ ]:
cat = np.loadtxt("../examples/COSMOS.in")

In [ ]:
zspec_mask = np.logical_and(cat[:, 62] > 0.01, cat[:, 62] < 6)
zs_cat = cat[zspec_mask]
spec = zs_cat[:, 0]
fluxes = zs_cat[:, 1:60:2]
efluxes = zs_cat[:, 2:61:2]
context = zs_cat[:, 61]
zspec = zs_cat[:, 62]

In [ ]:
keymap["ADAPT_LIM"], keymap["ADAPT_BAND"]

In [ ]:
mag = np.zeros_like(fluxes[:, 4]) + 1000.0
mag[fluxes[:, 4] > 0] = -2.5 * np.log10(fluxes[:, 4][fluxes[:, 4] > 0]) - 48.6
mag_mask = np.logical_and(mag > 1.5, mag < 20.5)
auto_adapt_data = zs_cat[mag_mask]

In [ ]:
srclist = []
nobj = 0
imagm = 30
gbcont = 0
contforb = -1
shifts0 = [0]
fac_err = [1.5]
min_err = [
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.05,
    0.05,
    0.05,
    0.05,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.02,
    0.05,
    0.05,
    0.05,
    0.05,
    0.1,
    0.2,
    0.3,
]
for d in auto_adapt_data:
    oneObj = onesource(nobj, photz.gridz)
    oneObj.readsource(str(d[0]), d[1:60:2], d[2:61:2], int(d[61]), d[62], str(d[-1]))
    photz.prep_data(oneObj)
    if len(shifts0) == imagm:
        oneObj.adapt_mag(shifts0, shifts1)
    srclist.append(oneObj)
    nobj += 1

In [ ]:
len(srclist)

In [ ]:
a0, a1 = photz.run_autoadapt(srclist)

In [ ]:
photozlist = []
for d in cat:
    oneObj = onesource(nobj, photz.gridz)
    oneObj.readsource(str(d[0]), d[1:60:2], d[2:61:2], int(d[61]), d[62], str(d[-1]))
    photz.prep_data(oneObj)
    photozlist.append(oneObj)

In [ ]:
len(photozlist)

In [ ]:
photz.run_photoz(photozlist[:100], a0, a1)

In [ ]:
t = photz.build_output_tables(photozlist[:100], para_out=None, filename="toto.fits")

In [ ]:
t